In [6]:
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from utils.readData import read_dataset
from utils.ResNet import ResNet18

# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 读数据
batch_size = 128
train_loader, valid_loader, test_loader = read_dataset(batch_size=batch_size, pic_path='dataset')
# 加载模型(使用预处理模型，修改最后一层，固定之前的权重)
n_class = 10
model = ResNet18()
"""
ResNet18网络的7x7降采样卷积和池化操作容易丢失一部分信息,
所以在实验中我们将7x7的降采样层和最大池化层去掉,替换为一个3x3的降采样卷积,
同时减小该卷积层的步长和填充大小
"""
model.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
model.fc = torch.nn.Linear(512, n_class)  # 将最后的全连接层改掉
model = model.to(device)
# 使用交叉熵损失函数
criterion = nn.CrossEntropyLoss().to(device)

# 开始训练
n_epochs = 10
valid_loss_min = np.Inf  # track change in validation loss
accuracy = []
lr = 0.1
counter = 0

for epoch in range(1, n_epochs + 1):
    print(f'Epoch {epoch}/{n_epochs}')
    
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    total_sample = 0
    right_sample = 0
    
    # 动态调整学习率
    if counter / 10 == 1:
        counter = 0
        lr = lr * 0.5
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    
    ###################
    # 训练集的模型 #
    ###################
    model.train()  # 启用batch normalization和drop out
    train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False)
    for data, target in train_loader_tqdm:
        data = data.to(device)
        target = target.to(device)
        # 清除梯度
        optimizer.zero_grad()
        # 正向传递：计算预测输出
        output = model(data).to(device)
        # 计算损失值
        loss = criterion(output, target)
        # 反向传递：计算损失相对于模型参数的梯度
        loss.backward()
        # 执行单个优化步骤（参数更新）
        optimizer.step()
        # 更新训练损失
        train_loss += loss.item() * data.size(0)
        
    ######################    
    # 验证集的模型 #
    ######################
    model.eval()  # 验证模型
    valid_loader_tqdm = tqdm(valid_loader, desc="Validation", leave=False)
    for data, target in valid_loader_tqdm:
        data = data.to(device)
        target = target.to(device)
        # 正向传递：计算预测输出
        output = model(data).to(device)
        # 计算损失值
        loss = criterion(output, target)
        # 更新验证损失
        valid_loss += loss.item() * data.size(0)
        # 将输出概率转换为预测类
        _, pred = torch.max(output, 1)
        # 将预测与真实标签进行比较
        correct_tensor = pred.eq(target.data.view_as(pred))
        total_sample += data.size(0)
        right_sample += correct_tensor.sum().item()
    
    # 计算准确率
    accuracy_epoch = 100 * right_sample / total_sample
    print(f"Accuracy: {accuracy_epoch}%")
    accuracy.append(accuracy_epoch)
 
    # 计算平均损失
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)
    
    # 显示训练集与验证集的损失函数
    print(f'Epoch: {epoch} \tTraining Loss: {train_loss:.6f} \tValidation Loss: {valid_loss:.6f}')
    
    # 如果验证集损失函数减少，就保存模型。
    if valid_loss <= valid_loss_min:
        print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint/resnet18_cifar10.pt')
        valid_loss_min = valid_loss
        counter = 0
    else:
        counter += 1


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10


Accuracy: 35.25%
Epoch: 1 	Training Loss: 2.049681 	Validation Loss: 1.799485
Validation loss decreased (inf --> 1.799485).  Saving model ...
Epoch 2/10


Accuracy: 41.61%
Epoch: 2 	Training Loss: 1.558186 	Validation Loss: 1.678528
Validation loss decreased (1.799485 --> 1.678528).  Saving model ...
Epoch 3/10


Accuracy: 51.88%
Epoch: 3 	Training Loss: 1.348117 	Validation Loss: 1.512662
Validation loss decreased (1.678528 --> 1.512662).  Saving model ...
Epoch 4/10


Accuracy: 57.54%
Epoch: 4 	Training Loss: 1.183386 	Validation Loss: 1.313750
Validation loss decreased (1.512662 --> 1.313750).  Saving model ...
Epoch 5/10


...

Accuracy: 69.71%
Epoch: 6 	Training Loss: 1.000067 	Validation Loss: 0.872880
Validation loss decreased (1.238933 --> 0.872880).  Saving model ...
Epoch 7/10


Accuracy: 68.58%
Epoch: 7 	Training Loss: 0.924699 	Validation Loss: 0.907587
Epoch 8/10


Accuracy: 72.26%
Epoch: 8 	Training Loss: 0.855978 	Validation Loss: 0.831369
Validation loss decreased (0.872880 --> 0.831369).  Saving model ...
Epoch 9/10


Accuracy: 73.85%
Epoch: 9 	Training Loss: 0.797777 	Validation Loss: 0.792299
Validation loss decreased (0.831369 --> 0.792299).  Saving model ...
Epoch 10/10


Accuracy: 74.53%
Epoch: 10 	Training Loss: 0.762155 	Validation Loss: 0.744260
Validation loss decreased (0.792299 --> 0.744260).  Saving model ...


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from utils.readData import read_dataset
from utils.ResNet import ResNet18
from tqdm import tqdm


device = 'cuda' if torch.cuda.is_available() else 'cpu'


batch_size = 128
train_loader, valid_loader, test_loader = read_dataset(batch_size=batch_size, pic_path='dataset')

# 加载模型
n_class = 10
model = ResNet18()
model.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
model.fc = torch.nn.Linear(512, n_class)  # 修改最后的全连接层
model = model.to(device)


criterion = nn.CrossEntropyLoss().to(device)


lr = 0.01 
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

# 训练
n_epochs = 10
valid_loss_min = float("inf") 
accuracy = []

for epoch in range(1, n_epochs + 1):
    print(f'Epoch {epoch}/{n_epochs}')
    
    # 训练和验证损失
    train_loss = 0.0
    valid_loss = 0.0
    total_sample = 0
    right_sample = 0

    ###################
    # 训练模型 #
    ###################
    model.train()
    train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False)
    for data, target in train_loader_tqdm:
        data = data.to(device)
        target = target.to(device)
        
        # 清除梯度
        optimizer.zero_grad()
        output = model(data).to(device)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        # 更新训练损失
        train_loss += loss.item() * data.size(0)
        
    ######################    
    # 验证模型 #
    ######################
    model.eval()
    valid_loader_tqdm = tqdm(valid_loader, desc="Validation", leave=False)
    for data, target in valid_loader_tqdm:
        data = data.to(device)
        target = target.to(device)
        
        # 正向传播
        output = model(data).to(device)
        

        loss = criterion(output, target)
        valid_loss += loss.item() * data.size(0)
        _, pred = torch.max(output, 1)
        
        correct_tensor = pred.eq(target.data.view_as(pred))
        total_sample += data.size(0)
        right_sample += correct_tensor.sum().item()
    
    # 计算acc
    accuracy_epoch = 100 * right_sample / total_sample
    print(f"Accuracy: {accuracy_epoch}%")
    accuracy.append(accuracy_epoch)
 

    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)

    print(f'Epoch: {epoch} \tTraining Loss: {train_loss:.6f} \tValidation Loss: {valid_loss:.6f}')
    

    if valid_loss <= valid_loss_min:
        print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), 'checkpoint/resnet18_cifar10.pt')
        valid_loss_min = valid_loss


model.load_state_dict(torch.load('checkpoint/resnet18_cifar10.pt'))

# 量化模型
model.eval()
model_int8 = torch.quantization.quantize_dynamic(
    model.cpu(),  
    {nn.Linear},  # 量化全连接层
    dtype=torch.qint8  # 量化为8位 int8
)

# 保存
torch.save(model_int8.state_dict(), 'checkpoint/resnet18_cifar10_int8.pt')


with open('checkpoint/resnet18_cifar10_int8.bin', 'wb') as f:
    for param_tensor in model_int8.state_dict().values():
        if isinstance(param_tensor, torch.Tensor):
            f.write(param_tensor.cpu().numpy().tobytes())

print("Model saved as binary file: 'checkpoint/resnet18_cifar10_int8.bin'")


Epoch 1/10


Accuracy: 42.16%
Epoch: 1 	Training Loss: 1.742501 	Validation Loss: 1.644033
Validation loss decreased (inf --> 1.644033). Saving model ...
Epoch 2/10


Accuracy: 51.05%
Epoch: 2 	Training Loss: 1.392996 	Validation Loss: 1.513441
Validation loss decreased (1.644033 --> 1.513441). Saving model ...
Epoch 3/10


Accuracy: 60.84%
Epoch: 3 	Training Loss: 1.196607 	Validation Loss: 1.120430
Validation loss decreased (1.513441 --> 1.120430). Saving model ...
Epoch 4/10


Accuracy: 66.19%
Epoch: 4 	Training Loss: 1.066062 	Validation Loss: 0.970725
Validation loss decreased (1.120430 --> 0.970725). Saving model ...
Epoch 5/10


...

Accuracy: 77.01%
Epoch: 7 	Training Loss: 0.814261 	Validation Loss: 0.676117
Validation loss decreased (0.786156 --> 0.676117). Saving model ...
Epoch 8/10


Accuracy: 75.91%
Epoch: 8 	Training Loss: 0.763166 	Validation Loss: 0.702353
Epoch 9/10


Accuracy: 78.94%
Epoch: 9 	Training Loss: 0.715948 	Validation Loss: 0.615700
Validation loss decreased (0.676117 --> 0.615700). Saving model ...
Epoch 10/10


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19908\2421495454.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('checkpoint/resne

Accuracy: 80.39%
Epoch: 10 	Training Loss: 0.677611 	Validation Loss: 0.591619
Validation loss decreased (0.615700 --> 0.591619). Saving model ...
Model saved as binary file: 'checkpoint/resnet18_cifar10_int8.bin'
